<a href="https://colab.research.google.com/github/saketkc/pyFLGLM/blob/master/Chapters/07_Chapter07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chapter 7 - Models for Count Data

In [ ]:
!pip install proplot

In [ ]:
import warnings

import pandas as pd
import proplot as plot
import seaborn as sns
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
from scipy import stats

warnings.filterwarnings("ignore")
%pylab inline


plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.weight"] = "bold"

Populating the interactive namespace from numpy and matplotlib


In [ ]:
cancer_df = pd.read_csv("https://github.com/saketkc/pyFLGLM/blob/master/data/Cancer.tsv.gz?raw=true", compression="gzip", sep="\t")
cancer_df.head()

,time,histology,stage,count,risktime
0,1,1,1,9,157
1,1,2,1,5,77
2,1,3,1,1,21
3,2,1,1,2,139
4,2,2,1,2,68


In [ ]:
cancer_df["logrisktime"] = np.log(cancer_df["risktime"])

formula = """count ~ C(histology) + C(stage) + C(time)"""
response, predictors = dmatrices(formula, cancer_df, return_type="dataframe")
fit = sm.GLM(
    response, predictors, family=sm.families.Poisson(link=sm.families.links.log()),
    offset=cancer_df["logrisktime"]
).fit()
print(fit.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                  count   No. Observations:                   63
Model:                            GLM   Df Residuals:                       52
Model Family:                 Poisson   Df Model:                           10
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -114.87
Date:                Sun, 09 Aug 2020   Deviance:                       43.923
Time:                        20:35:58   Pearson chi2:                     43.1
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -3.0093      0.16

The increasing coefficients with stage reflect the higher mortality with stage. Stage 3 mortalites are $exp(1.324) = 3.76$ times higher than staeg 1.

In [ ]:
drugs_df = pd.read_csv("https://github.com/saketkc/pyFLGLM/blob/master/data/Drugs.tsv.gz?raw=true", compression="gzip", sep="\t")
drugs_df = drugs_df.rename(columns={"A": "alc", "C": "cig", "M": "mar"})
drugs_df

,alc,cig,mar,count
0,yes,yes,yes,911
1,yes,yes,no,538
2,yes,no,yes,44
3,yes,no,no,456
4,no,yes,yes,3
5,no,yes,no,43
6,no,no,yes,2
7,no,no,no,279


In [ ]:
formula = """count ~ C(alc) + C(cig) + C(mar)"""
response, predictors = dmatrices(formula, drugs_df, return_type="dataframe")
mutual_indep = sm.GLM(
    response, predictors, family=sm.families.Poisson(link=sm.families.links.log())).fit()
print(mutual_indep.summary())



                 Generalized Linear Model Regression Results                  
Dep. Variable:                  count   No. Observations:                    8
Model:                            GLM   Df Residuals:                        4
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -667.53
Date:                Sun, 09 Aug 2020   Deviance:                       1286.0
Time:                        20:54:10   Pearson chi2:                 1.41e+03
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         4.1725      0.065     64.234

In [ ]:
l = ["yes", "no"]
formula = """count ~ C(alc, levels=l) + C(cig, levels=l) + C(mar, levels=l) + C(alc, levels=l):C(cig, levels=l) + C(alc, levels=l):C(mar,levels=l) + C(cig,levels=l):C(mar,levels=l)"""
response, predictors = dmatrices(formula, drugs_df, return_type="dataframe")
homo_association = sm.GLM(
    response, predictors, family=sm.families.Poisson(link=sm.families.links.log())).fit()

print(homo_association.summary())
print('AIC: {}'.format(homo_association.aic))
pearson_resid = homo_association.resid_pearson
std_resid = homo_association.resid_response
print(np.sum(pearson_resid**2))

counts = drugs_df["count"]

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  count   No. Observations:                    8
Model:                            GLM   Df Residuals:                        1
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -24.709
Date:                Sun, 09 Aug 2020   Deviance:                      0.37399
Time:                        21:49:27   Pearson chi2:                    0.401
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [ ]:
df = pd.DataFrame( np.vstack([counts.values,
                              homo_association.fittedvalues, 
                              homo_association.resid_pearson, 
                              homo_association.resid_response])).T
df.columns = ["count", "fitted", "pearsonr_resid", "std_resid"]
df


,count,fitted,pearsonr_resid,std_resid
0,911.0,910.38317,0.020443,0.61683
1,538.0,538.61683,-0.026578,-0.61683
2,44.0,44.61683,-0.092346,-0.61683
3,456.0,455.38317,0.028905,0.61683
4,3.0,3.61683,-0.324341,-0.61683
5,43.0,42.38317,0.094748,0.61683
6,2.0,1.38317,0.524479,0.61683
7,279.0,279.61683,-0.036888,-0.61683


In [ ]:
drugs2_df = pd.read_csv("https://github.com/saketkc/pyFLGLM/blob/master/data/Drugs2.tsv.gz?raw=true", compression="gzip", sep="\t")
drugs2_df = drugs2_df.rename(columns={"A": "alc", "C": "cig"})
drugs2_df["M_yes_byn"] = drugs2_df["M_yes"]/drugs2_df["n"]

In [ ]:
l = ["yes", "no"]

#formula = """M_yes/n ~ C(alc, levels=l) + C(cig, levels=l)"""
#formula = """I(M_yes/n) ~ C(alc) + C(cig)"""
formula = """M_yes_byn ~ C(alc) + C(cig)"""

response, predictors = dmatrices(formula, drugs2_df, return_type="dataframe")
fit = sm.GLM(response, 
             predictors, 
             family=sm.families.Binomial(link=sm.families.links.logit()),
             weights=drugs2_df["n"]).fit()
print(fit.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:              M_yes_byn   No. Observations:                    4
Model:                            GLM   Df Residuals:                        1
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:               -0.86889
Date:                Sun, 09 Aug 2020   Deviance:                    0.0017527
Time:                        22:01:57   Pearson chi2:                  0.00204
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -5.4367      5.314     -1.023

### Section 7.5.1

In [ ]:
crabs_df = pd.read_csv("https://github.com/saketkc/pyFLGLM/blob/master/data/Crabs.tsv.gz?raw=true", compression="gzip", sep="\t")
crabs_df.head()

,crab,y,weight,width,color,spine
1,1,8,3.05,28.3,2,3
2,2,0,1.55,22.5,3,3
3,3,9,2.30,26.0,1,1
4,4,0,2.10,24.8,3,3
5,5,4,2.60,26.0,3,3


In [ ]:
formula = """y ~ 1"""

response, predictors = dmatrices(formula, crabs_df, return_type="dataframe")
fit = sm.GLM(response, 
             predictors, 
             family=sm.families.Poisson(link=sm.families.links.log())).fit()
print(fit.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  173
Model:                            GLM   Df Residuals:                      172
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -494.04
Date:                Sun, 09 Aug 2020   Deviance:                       632.79
Time:                        22:06:20   Pearson chi2:                     584.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0713      0.044     24.074      0.0

In [ ]:
formula = """y ~ 1"""

response, predictors = dmatrices(formula, crabs_df, return_type="dataframe")
fit = sm.GLM(response, 
             predictors, 
             family=sm.families.NegativeBinomial(link=sm.families.links.log())).fit(scale='x2')
#  (crabs_df["y"].var()-crabs_df["y"].mean())/(crabs_df["y"].mean()**2)#- fit.mu             
overdispersion = fit.pearson_chi2 / fit.df_resid
print(fit.summary())

print('Overdispersion: {}'.format(overdispersion))


                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  173
Model:                            GLM   Df Residuals:                      172
Model Family:        NegativeBinomial   Df Model:                            0
Link Function:                    log   Scale:                         0.86643
Method:                          IRLS   Log-Likelihood:                -444.42
Date:                Sun, 09 Aug 2020   Deviance:                       224.93
Time:                        22:52:01   Pearson chi2:                     149.
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0713      0.082     13.064      0.0

In [ ]:
import statsmodels.discrete.count_model as cm
formula = """y ~ 1"""

response, predictors = dmatrices(formula, crabs_df, return_type="dataframe")

fit = cm.ZeroInflatedPoisson(response, 
                             predictors).fit()
print(fit.summary())

Optimization terminated successfully.
         Current function value: 2.205865
         Iterations: 6
         Function evaluations: 8
         Gradient evaluations: 8
                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:                       y   No. Observations:                  173
Model:             ZeroInflatedPoisson   Df Residuals:                      172
Method:                            MLE   Df Model:                            0
Date:                 Sun, 09 Aug 2020   Pseudo R-squ.:               2.817e-11
Time:                         23:33:48   Log-Likelihood:                -381.61
converged:                        True   LL-Null:                       -381.61
Covariance Type:             nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const    -0.6139   

In [ ]:
import statsmodels.discrete.count_model as cm
formula = """y ~ 1"""

response, predictors = dmatrices(formula, crabs_df, return_type="dataframe")

fit = cm.ZeroInflatedNegativeBinomialP(response, 
                                                           predictors, 
                                                           p=2).fit()
print(fit.summary())

Optimization terminated successfully.
         Current function value: 2.134980
         Iterations: 12
         Function evaluations: 13
         Gradient evaluations: 13
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:                                 y   No. Observations:                  173
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                      172
Method:                                      MLE   Df Model:                            0
Date:                           Sun, 09 Aug 2020   Pseudo R-squ.:               3.957e-11
Time:                                   23:35:03   Log-Likelihood:                -369.35
converged:                                  True   LL-Null:                       -369.35
Covariance Type:                       nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
--------------------------

In [ ]:
formula = """y ~ weight + color"""

response, predictors = dmatrices(formula, crabs_df, return_type="dataframe")

fit = cm.ZeroInflatedNegativeBinomialP(response, 
                                       predictors).fit()
print(fit.summary())

Optimization terminated successfully.
         Current function value: 2.115471
         Iterations: 19
         Function evaluations: 22
         Gradient evaluations: 22
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:                                 y   No. Observations:                  173
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                      170
Method:                                      MLE   Df Model:                            2
Date:                           Sun, 09 Aug 2020   Pseudo R-squ.:                0.009138
Time:                                   23:41:48   Log-Likelihood:                -365.98
converged:                                  True   LL-Null:                       -369.35
Covariance Type:                       nonrobust   LLR p-value:                   0.03421
                    coef    std err          z      P>|z|      [0.025      0.975]
--------------------------

In [ ]:
formula = """y ~ weight + color"""

response, predictors = dmatrices(formula, crabs_df, return_type="dataframe")

fit = sm.GLM(response, 
             predictors, 
             family=sm.families.NegativeBinomial(link=sm.families.links.log())).fit(scale='x2')
print(fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  173
Model:                            GLM   Df Residuals:                      170
Model Family:        NegativeBinomial   Df Model:                            2
Link Function:                    log   Scale:                         0.95361
Method:                          IRLS   Log-Likelihood:                -391.41
Date:                Sun, 09 Aug 2020   Deviance:                       201.33
Time:                        23:45:50   Pearson chi2:                     162.
No. Iterations:                    15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3177      0.532     -0.597      0.5